<a href="https://colab.research.google.com/github/nestorbalcazar/nestorbalcazar.github.io/blob/master/Rad_RadiosityMethod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ==========================================================================
### Subject:    Heat and mass Transfer
### Topic:      Radiative transfer.
### Author:     Prof. Nestor Balcazar Arciniega
### Updated:    5 November 2024
### ==========================================================================

In [42]:
# ==========================================================================
# Subject:    Heat and mass Transfer
# Topic:      Radiative transfer. Problem 4.
# Author:     Nestor V. Balcazar Arciniega
# Updated:    8 November 2024
# ==========================================================================
# Version:    Python 3.x
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.special import iv, kv
from scipy.integrate import quad

# ==========================================================================
# INPUT PARAMETERS
sigma = 5.67e-8                    # Constante de Stefan-Boltzmann
delta = 1e-13                      # Tolerancia para el criterio de convergencia

eps   = np.array([0.9, 1, 0.7, 1]) #Emissivity
T     = np.array([250+273.15, 25+273.15, 50+273.15, 25+273.15])   #Temperature

# Definir el vector posición de los puntos del contorno cerrado en el sentido de giro del reloj
x = np.array([
    [0, 0],
    [0, 1.5],
    [0.5 + 1.5 * np.cos(65 * (np.pi / 180)), 1.5],
    [0.5, 0]
])

#VIEW FACTORS
# Número de puntos en el contorno
Ni = x.shape[0]

# Inicializar la matriz de factores de forma
F = np.zeros((Ni, Ni))

# Función para calcular la distancia euclidiana entre dos puntos
def distance(p1, p2):
    return np.linalg.norm(p1 - p2)

# Calcular los factores de forma utilizando el método de Hottel
for i in range(Ni):
    for j in range(Ni):
        if i != j:  # No calculamos el factor de forma de un segmento consigo mismo
            # Puntos del segmento i
            p1_i = x[i]
            p2_i = x[(i + 1) % Ni]

            # Puntos del segmento j
            p1_j = x[j]
            p2_j = x[(j + 1) % Ni]

            # Distancias entre puntos
            d4 = distance(p1_i, p2_j)
            d3 = distance(p2_i, p1_j)
            d2 = distance(p1_i, p1_j)
            d1 = distance(p2_i, p2_j)

            # Longitud del segmento i
            Li = distance(p1_i, p2_i)

            # Factor de forma entre los segmentos i y j
            F[i, j] = ( (d1 + d2) - (d3 + d4) ) / (2 * Li)

# Imprimir la matriz de factores de forma
print("Matriz de Factores de Forma (Fij):")
print(F)
# ==========================================================================
# Valores iniciales de j0 basados en las temperaturas iniciales
j  = eps * sigma * (T ** 4)          # Calcula j0 usando la ecuación de cuerpo negro
j0 = j.copy()                       # Copia inicial de j para iteración

# Bucle de iteración
while True:
    for i in range(Ni):
        RHS = 0
        for k in range(Ni):  # Cambiamos el índice de bucle interno a `k`
            if i != k:  # Condición para excluir i == k
                RHS += (1 - eps[i]) * F[i, k] * j0[k]  # Acumulación del término RHS

        # Actualización de j(i) según la fórmula
        j[i] = (1.0 / (1 - (1 - eps[i]) * F[i, i])) * (RHS + eps[i] * sigma * T[i] ** 4)

    # Cálculo del error y comprobación del criterio de convergencia
    error = np.linalg.norm(j - j0)  # Norma de la diferencia entre j actual y j previo
    if error < delta:
        break  # Condición de salida del bucle si el error es menor que delta

    # Actualización de j0 para la siguiente iteración
    j0 = j.copy()

# Mostrar resultado final
print("Valores finales de j:", j)

g = np.sum(F * j, axis=1)

# Calcular q como la diferencia entre j y g
q = j - g

# Mostrar los resultados
print("g:", g)
print("q:", q)

# Area of the surfaces
# Inicializar el vector S
S = np.zeros(Ni)
for i in range(Ni):
    S[i] = np.linalg.norm(x[i, :] - x[(i + 1) % Ni, :])

# Calcular res_qradS como el producto escalar entre q y S
res_qradS = np.sum(q * S)

# Mostrar el resultado
print("S =", S)
print("res_qradS = sum(q * S) =", res_qradS)

Matriz de Factores de Forma (Fij):
[[0.         0.2511858  0.60919381 0.13962039]
 [0.33227762 0.         0.52086361 0.14685877]
 [0.56113987 0.36268838 0.         0.07617175]
 [0.41886117 0.33305437 0.24808446 0.        ]]
Valores finales de j: [3909.91765452  448.04569997 1150.00247045  448.04569997]
g: [ 875.67342086 1963.97201517 2390.64006192 2072.2340066 ]
q: [ 3034.24423366 -1515.92631521 -1240.63759146 -1624.18830663]
S = [1.5        1.13392739 1.62845446 0.5       ]
res_qradS = sum(q * S) = -4.547473508864641e-13
